# Data Science for Business

## Spring 2020, module 4 @ HSE

---

## Home assignment 5


Author: **Miron Rogovets**

---

You goal for this task is two fold:

1. Cluster all the products into distinct groups (clusters)
2. Build a recommender system for customers, but instead of products we will recommend categories.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
sns.set_style('darkgrid')

In [5]:
df = pd.read_csv('data/sample_transations.csv', index_col=0)
df.head(3)

,dd_card_number,store_number,dd_transaction_number,sku,quantity,post_discount_price,date,hour,dbi_item_catgry,dbi_item_sub_catgry,dbi_item_famly_name,dbi_item_size
0,0,775,7969,74,1,2.29,9/2/2015,Lunch,Beverages,Hot Coffee,Hot Coffee,Large
1,0,775,7969,73,1,0.00,9/2/2015,Lunch,Beverages,Hot Coffee,Hot Coffee,Medium
2,0,761,16021,75,1,2.49,9/4/2015,Morning,Beverages,Hot Coffee,Hot Coffee,X-Large


In [6]:
df.shape

(51939, 12)

In [7]:
df.dtypes

dd_card_number             int64
store_number               int64
dd_transaction_number      int64
sku                        int64
quantity                   int64
post_discount_price      float64
date                      object
hour                      object
dbi_item_catgry           object
dbi_item_sub_catgry       object
dbi_item_famly_name       object
dbi_item_size             object
dtype: object

In [9]:
df.isna().sum()

dd_card_number           0
store_number             0
dd_transaction_number    0
sku                      0
quantity                 0
post_discount_price      0
date                     0
hour                     0
dbi_item_catgry          0
dbi_item_sub_catgry      0
dbi_item_famly_name      0
dbi_item_size            0
dtype: int64

- `dd_card_number` - customer identifier (categorical)
- `store_number` - store identifier (categorical)
- `dd_transaction_number` - transaction identifier (categorical)
- `sku` - product identifier, may vary across different stores (categorical)
- `quantity` - quantity (numerical)
- `post_discount_price` - transaction price (numerical)
- `date` - transaction date 
- `hour` (categorical)
- `dbi_item_catgry` (categorical)
- `dbi_item_sub_catgry` (categorical)
- `dbi_item_famly_name` (categorical)
- `dbi_item_size` (categorical)


### I. Clustering (20)

1. Feature generation. Use examples from Seminar 6 Plan to generate features for products clustering. You may generate any number of features but you must generate at least 3 features which differ from those, proposed in the plan.

2. Cluster all products into distinct groups (clusters). You may use any clustering algorithm you want. If you use distance-based clustering (e.g. k-means), do not forget to preprocess your features (normalization, z-scoring or standard scaling). Try a different number of groups (e.g. from 5 to 30)

3. Write a report. In your report you should present the following information:
   - Put an example screenshot of your features.
   - Explain (in a similar way I explain them in the plan) every single feature (you may skip features from the seminar plan) you use.
   - Cluster’s information: how many clusters do you have, how many objects are in these clusters.
   - Cluster’s interpretation. Try to provide an interpretation of every single cluster (or groups of clusters) you end up. For example: “Cluster 1 includes hot drinks and beverages often bought in a combination in the morning.”
   - You may include any visualization you find necessary, e.g.: colored PCA components, histogram or pie charts of cluster’s sizes, “elbows” used for selection number of clusters (if you have used it).


### II. Recommender system (25)

1. Prepare user-item data as it was done during the seminar: User, Item, Score. You may construct Score (e.g. see seminar) any way you want, but you must explain it in your report.  

2. Split your data into train and test sets (as Leonid explained during the lecture): some of the user-item pairs go to the train set and some to the test set.


3. Build a recommender system using cluster groups (if you have about 20-40 clusters) or items subcategories (75 most frequent values of the  `dbi_item_famly_name`  attribute) as items and `dd_card_number` as users. You may want to play with a number of neighbours in your KNN recommender model. 

4. Compute 3 different recommender performance scores, which were explained during the lecture or seminar to assess the quality of your recommendations (use appropriate metrics).

5. Write a report.  In your report you should present the following information:
   - Report computed performance scores.
   - Elaborate on the quality of your recommendations.
   - Provide 3-5 examples of `good` recommendations suggested by your recommender system.
   - Provide 3-5 examples of `bad` recommendations suggested by your recommender system.
   - You may report any additional information you find potentially useful to assess the quality of your recommendations: e.g for a couple of customers compute the price of their average purchase (or an item in purchase) and compare it with the average price of recommended items.
   - You may use any visualisations you find useful
